In [1]:
!wget http://yaroslavvb.com/upload/notMNIST/notMNIST_large.tar.gz

--2021-02-06 08:02:26--  http://yaroslavvb.com/upload/notMNIST/notMNIST_large.tar.gz
Resolving yaroslavvb.com (yaroslavvb.com)... 129.121.4.193
Connecting to yaroslavvb.com (yaroslavvb.com)|129.121.4.193|:80... connected.
HTTP request sent, awaiting response...200 OK
Length: 247336696 (236M) [application/x-gzip]
Saving to: ‘notMNIST_large.tar.gz’

notMNIST_large.tar. 100%[===================>] 235.88M  37.4MB/s    in 6.5s    

2021-02-06 08:02:33 (36.4 MB/s) - ‘notMNIST_large.tar.gz’ saved [247336696/247336696]



In [2]:
!gunzip notMNIST_large.tar.gz

In [ ]:
import os
os.getcwd()